In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.datasets as datasets
from torchvision.utils import make_grid
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

In [2]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [48]:
#www.bike.bike

#initialize input (limited to the first 1000 values)
input = F.interpolate(mnist_trainset.data.unsqueeze(1).float(), size=(32,32))[:5000]

#construction target
target = torch.zeros(len(input),10)
value = mnist_trainset.targets[:len(input)]
for ind in range(len(target)):
    target[ind,value[ind]] = 1
    
optimizer = optim.SGD(net.parameters(), lr=0.02)
criterion = nn.MSELoss()

In [49]:
for ind in range(50):
    #calcul output
    output = net(input)

    #calcul loss
    loss = criterion(output, target)
    print('{}: {}'.format(ind,loss.item()))

    #backprop
    net.zero_grad()
    loss.backward()
    optimizer.step()
    
    learning_rate = 0.02
    
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

0: 0.02049335464835167
1: 0.020481381565332413
2: 0.02046944573521614
3: 0.020457550883293152
4: 0.020445674657821655
5: 0.020433802157640457
6: 0.020421970635652542
7: 0.020410191267728806
8: 0.020398437976837158
9: 0.020386703312397003
10: 0.020374992862343788
11: 0.020363319665193558
12: 0.020351670682430267
13: 0.02034006081521511
14: 0.020328463986516
15: 0.020316893234848976
16: 0.02030535601079464
17: 0.020293833687901497
18: 0.020282335579395294
19: 0.02027086727321148
20: 0.02025943621993065
21: 0.020248007029294968
22: 0.02023659646511078
23: 0.020225221291184425
24: 0.02021387778222561
25: 0.02020256593823433
26: 0.020191282033920288
27: 0.02018003724515438
28: 0.02016882784664631
29: 0.020157644525170326
30: 0.02014647237956524
31: 0.02013532817363739
32: 0.02012421190738678
33: 0.020113132894039154
34: 0.020102081820368767
35: 0.020091049373149872
36: 0.020080044865608215
37: 0.020069070160388947
38: 0.020058119669556618
39: 0.020047185942530632
40: 0.020036274567246437
41

In [44]:
input = F.interpolate(mnist_testset.data.unsqueeze(1).float(), size=(32,32))
output = net(input)

In [45]:
comp = torch.argmax(output,axis=1) == mnist_testset.targets

In [46]:
torch.sum(comp).item()/len(comp)*100

91.8

In [53]:
import numpy as np
cam = np.eye(22)
print(cam)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.